**참고문헌:  딥 러닝을 이용한 자연어 처리 입문, 11장- NLP를 위한 합성곱 신경망**

합성곱 신경망은 주로 이미지 처리. 즉, 비전 분야에서 사용되는 알고리즘이지만 이를 응용해서 자연어 처리에 사용하기 위한 시도들이 있었습니다. 이번 챕터에서는 비전 분야에서 사용되는 합성곱 신경망의 동작 방식에 대해서 이해하고, 이 개념을 확장하여 자연어 처리를 위한 1D 합성곱 신경망을 이해합니다. 그리고앞서 RNN으로 텍스트 분류를 수행했던 데이터들을 가지고 합성곱 신경망을 통해서 분류 문제를 풀어봅니다. 추가적으로 합성곱 신경망을 이용하여 문자 임베딩(Character Embedding)을 얻는 방법에 대해서 설명합니다.


# 자연어 처리를 위한 1D CNN(1D Convolutional Neural Networks)

## 1. 2D 합성곱(2D Convolutions)


앞서 합성곱 신경망을 설명하며 합성곱 연산을 다음과 같이 정의했습니다.


> 합성곱 연산이란 커널(kernel) 또는 필터(filter) 라는 n × m 크기의 행렬로 높이(height) × 너비(width) 크기의 이미지를 처음부터 끝까지 겹치며 훑으면서 n × m 크기의 겹쳐지는 부분의 각 이미지와 커널의 원소의 값을 곱해서 모두 더한 값을 출력으로 하는 것을 말합니다. 이때, 이미지의 가장 왼쪽 위부터 가장 오른쪽 아래까지 순차적으로 훑습니다.

위와 같은 이미지 처리에서의 합성곱 연산을 2D 합성곱 연산이라고 부릅니다.


## 2. 1D 합성곱(1D Convolutions)

자연어 처리에 사용되는 1D 합성곱 연산을 정리해봅시다. LSTM을 이용한 여러 실습을 상기해보면, 각 문장은 임베딩 층(embedding layer)을 지나서 각 단어가 임베딩 벡터가 된 상태로 LSTM의 입력이 되었습니다. 이는 1D 합성곱 연산에서도 마찬가지입니다. 1D 합성곱 연산에서도 입력이 되는 것은 각 단어가 벡터로 변환된 문장 행렬로 LSTM과 입력을 받는 형태는 동일합니다.

'wait for the video and don't rent it'이라는 문장이 있을 때, 이 문장이 토큰화, 패딩, 임베딩 층(Embedding layer)을 거친다면 다음과 같은 문장 형태의 행렬로 변환될 것입니다. 아래 그림에서 은 문장의 길이, 는 임베딩 벡터의 차원입니다.

<img src = "https://wikidocs.net/images/page/80437/sentence_matrix.PNG">

그리고 이 행렬이 만약 LSTM의 입력으로 주어진다면, LSTM은 첫번째 시점에는 첫번째 행을 입력으로 받고, 두번째 시점에는 두번째 행을 입력으로 받으며 순차적으로 단어를 처리합니다. 그렇다면 1D 합성곱 연산의 경우에는 저 행렬을 어떻게 처리할까요?

1D 합성곱 연산에서 커널의 너비는 문장 행렬에서의 임베딩 벡터의 차원과 동일하게 설정됩니다. 그렇기 때문에 1D 합성곱 연산에서는 커널의 높이만으로 해당 커널의 크기라고 간주합니다. 가령, 커널의 크기가 2인 경우에는 아래의 그림과 같이 높이가 2, 너비가 임베딩 벡터의 차원인 커널이 사용됩니다.

<img src = "https://wikidocs.net/images/page/80437/1d_cnn.PNG">

커널의 너비가 임베딩 벡터의 차원이라는 의미는 커널이 2D 합성곱 연산때와는 달리 너비 방향으로는 더 이상 움직일 곳이 없다는 것을 의미합니다. 그래서 1D 합성곱 연산에서는 커널이 문장 행렬의 높이 방향으로만 움직이게 되어있습니다. 쉽게 설명하면, 위 그림에서 커널은 2D 합성곱 연산때와는 달리 오른쪽으로는 움직일 공간이 없으므로, 아래쪽으로만 이동해야 합니다.



한 번의 연산을 1 스텝(step)이라고 하였을 때, 합성곱 연산의 네번째 스텝까지 표현한 이미지는 다음과 같습니다. 크기가 2인 커널은 처음에는 'wait for'에 대해서 합성곱 연산을 하고, 두번째 스텝에는 'for the'에 대해서 연산을, 세번째 스텝에는 'the video'에 대해서 연산을, 네번째 스텝에서는 'video and'에 대해서 연산을 하게 됩니다.

<img src = "https://wikidocs.net/images/page/80437/%EB%84%A4%EB%B2%88%EC%A7%B8%EC%8A%A4%ED%85%9D.PNG">

이렇게 여덟번째 스텝까지 반복하였을 때, 결과적으로는 우측의 8차원 벡터를 1D 합성곱 연산의 결과로서 얻게될 것입니다. 그런데 커널의 크기가 꼭 2일 필요가 있을까요? 2D 합성곱 연산에서 커널의 크기가 3 × 3 또는 5 × 5 또는 등등의 여러 크기의 커널을 자유자재로 사용할 수 있었듯이, 1D 합성곱 연산에서도 커널의 크기는 사용자가 변경할 수 있습니다. 가령, 커널의 크기를 3으로 한다면, 네번째 스텝에서의 연산은 아래의 그림과 같을 것입니다.

<img src = "https://wikidocs.net/images/page/80437/%EC%BB%A4%EB%84%903.PNG">

커널의 크기가 달라진다는 것은 어떤 의미가 있을까요? CNN에서의 커널은 신경망 관점에서는 가중치 행렬이므로 커널의 크기에 따라 학습하게 되는 파라미터의 수는 달라집니다. 1D 합성곱 연산과 자연어 처리 관점에서는 커널의 크기에 따라서 참고하는 단어의 묶음의 크기가 달라집니다. 이는 참고하는 n-gram이 달라진다고 볼 수 있습니다. 커널의 크기가 2라면 각 연산의 스텝에서 참고하는 것은 bigram입니다. 커널의 크기가 3이라면 각 연산의 스텝에서 참고하는 것은 trigram입니다.



## 3. 맥스 풀링(Max-pooling)

이미지 처리에서의 CNN에서 그랬듯이, 일반적으로 1D 합성곱 연산을 사용하는 1D CNN에서도 합성곱 층(합성곱 연산 + 활성화 함수) 다음에는 풀링 층을 추가하게됩니다. 그 중 대표적으로 사용되는 것이 맥스 풀링(Max-pooling)입니다. 맥스 풀링은 각 합성곱 연산으로부터 얻은 결과 벡터에서 가장 큰 값을 가진 스칼라 값을 빼내는 연산입니다.

아래의 그림은 크기가 2인 커널과 크기가 3인 커널 두 개의 커널로부터 각각 결과 벡터를 얻고, 각 벡터에서 가장 큰 값을 꺼내오는 맥스 풀링 연산을 보여줍니다.

<img src = "https://wikidocs.net/images/page/80437/%EB%A7%A5%EC%8A%A4%ED%92%80%EB%A7%81.PNG">

## 4. 신경망 설계하기

지금까지 배운 개념들을 가지고 텍스트 분류를 위한 CNN을 설계해봅시다. 우선, 설계하고자 하는 신경망은 이진 분류를 위한 신경망입니다. 단, 시그모이드 함수가 아니라 소프트맥스 함수를 사용할 것이므로 출력층에서 뉴런의 개수가 2인 신경망을 설계합니다.

<img src = "https://wikidocs.net/images/page/80437/conv1d.PNG">

커널은 크기가 4인 커널 2개, 3인 커널 2개, 2인 커널 2개를 사용합니다. 문장의 길이가 9인 경우, 합성곱 연산을 한 후에는 각각 6차원 벡터 2개, 7차원 벡터 2개, 8차원 벡터 2개를 얻습니다. 벡터가 6개므로 맥스 풀링을 한 후에는 6개의 스칼라 값을 얻는데, 일반적으로 이렇게 얻은 스칼라값들은 전부 연결(concatenate)하여 하나의 벡터로 만들어줍니다. 이렇게 얻은 벡터는 1D CNN을 통해서 문장으로부터 얻은 벡터입니다. 이를 뉴런이 2개인 출력층에 완전 연결시키므로서(Dense layer를 사용) 텍스트 분류를 수행합니다.

## 5. 케라스(Keras)로 CNN 구현하기


케라스로 1D 합성곱 층을 추가하는 코드는 다음과 같습니다.


```
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
```



각 인자에 대한 설명은 다음과 같습니다.

num_filters = 커널의 개수.

kernel_size = 커널의 크기.

padding = 패딩 방법.

- valid : 패딩 없음. 제로 패딩없이 유효한(valid) 값만을 사용한다는 의미에서 valid.

- same : 합성곱 연산 후에 출력이 입력과 동일한 차원을 가지도록 왼쪽과 오른쪽(또는 위, 아래)에 제로 패딩을 추가.

activation = 활성화 함수.



만약 위에서 설명한 맥스 풀링을 추가하고자 한다면 다음과 같이 코드를 작성할 수 있습니다.

```
model = Sequential()
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
```

# 1D CNN으로 IMDB 리뷰 분류하기

1D CNN을 이용하여 IMDB 리뷰를 분류해보겠습니다. 이전에 다룬 데이터이므로 데이터에 대한 설명은 생략합니다.



## 1. 데이터에 대한 전처리


In [ ]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences

최대 10,000개의 단어만을 허용하여 데이터를 로드합니다.

In [ ]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words=vocab_size)

X_train을 상위 5개만 출력해봅시다.

In [ ]:
print(X_train[:5])

각 샘플의 길이가 긴 관계로 출력 시 중간 내용은 중략하였습니다. 각 샘플은 이미 정수 인코딩까지 전처리가 된 상태입니다. 하지만 각 샘플들의 길이는 서로 다르죠? 패딩을 진행하여 모든 샘플들의 길이를 200으로 맞춥니다.

In [ ]:
max_len = 200
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

패딩이 되었는지 크기(shape)를 확인해봅시다.

In [ ]:
print('X_train의 크기(shape) :',X_train.shape)
print('X_test의 크기(shape) :',X_test.shape)

훈련 데이터, 테스트 데이터 각 25,000 샘플이 전부 길이 200을 가지는 것을 확인할 수 있습니다. y_train도 출력해봅시다.

In [ ]:
print(y_train[:5])

1과 0으로 구성된 것을 확인하였습니다. 이진 분류를 수행할 것이므로 레이블에는 더 이상 전처리를 할 것이 없습니다.

## 2. 1D CNN으로 IMDB 리뷰 분류하기


IMDB 리뷰 분류를 위한 1D CNN 모델을 설계해봅시다. 하이퍼파라미터인 임베딩 벡터의 차원은 256, 드롭 아웃 비율은 0.3, 커널의 크기는 3이며 해당 커널은 총 256개 사용합니다. 합성곱 층과 맥스풀링 연산 후 전결합층(Fully Connected Layer)을 은닉층을 추가로 사용했는데, 은닉층의 뉴런 수는 128입니다. 해당 모델은 마지막 시점에서 두 개의 선택지 중 하나를 예측하는 이진 분류 문제를 수행하는 모델입니다. 이진 분류 문제의 경우, 출력층에 로지스틱 회귀를 사용해야 하므로 활성화 함수로는 시그모이드 함수를 사용하고, 손실 함수로 크로스 엔트로피 함수를 사용합니다. 20 에포크를 수행합니다.

EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)는 검증 데이터 손실(val_loss)이 증가하면, 과적합 징후므로 검증 데이터 손실이 3회 증가하면 정해진 에포크가 도달하지 못하였더라도 학습을 조기 종료(Early Stopping)한다는 의미입니다. ModelCheckpoint를 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델을 저장합니다. validation_data로는 X_test와 y_test를 사용합니다. val_loss가 줄어들다가 증가하는 상황이 오면 과적합으로 판단하기 위함입니다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 256 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[es, mc])

저자의 경우 에포크 4에서 조기 종료되었습니다. 저장된 모델을 로드하여 테스트 정확도를 확인합니다.

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

# Multi-Kernel 1D CNN으로 네이버 영화 리뷰 분류하기


## 1. 네이버 영화 데이터 수집 & 전처리

모든 전처리는 RNN을 이용한 텍스트 분류 챕터의 네이버 영화 리뷰 분류하기와 동일하게 수행하였다고 가정합니다.



## 2. Multi-Kernel 1D CNN으로 네이버 영화 리뷰 분류하기
케라스에서 다수의 커널을 사용할 경우에는 Funtional API를 사용하여 구현합니다. 우선 필요한 도구들을 임포트합니다. 하이퍼파라미터인 임베딩 벡터의 차원은 128, 드롭아웃 비율은 0.5와 0.8 두 가지를 사용합니다. 각 커널의 개수는 128개를 사용하고, 전결합층(Fully Connected Layer)을 은닉층을 추가로 사용했는데, 은닉층의 뉴런 수는 128입니다.

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, Input, Flatten, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 128
dropout_ratio = (0.5, 0.8)
num_filters = 128
hidden_units = 128

입력 층과 임베딩 층을 정의합니다. 임베딩 층 이후에는 드롭아웃의 인자값이 0.5. 즉, 50% 드롭아웃을 해주었습니다.

In [ ]:
model_input = Input(shape = (max_len,))
z = Embedding(vocab_size, embedding_dim, input_length = max_len, name="embedding")(model_input)
z = Dropout(dropout_ratio[0])(z)

3, 4, 5의 크기를 가지는 커널을 각각 128개 사용합니다. 그리고 이들을 맥스풀링합니다.

In [ ]:
conv_blocks = []

for sz in [3, 4, 5]:
    conv = Conv1D(filters = num_filters,
                         kernel_size = sz,
                         padding = "valid",
                         activation = "relu",
                         strides = 1)(z)
    conv = GlobalMaxPooling1D()(conv)
    conv_blocks.append(conv)

각 맥스풀링한 결과를 연결(concatenate)합니다. 그리고 이를 전결합층(Fully Connected Layer)을 사용한 은닉층으로 전달합니다. 해당 모델은 마지막 시점에서 두 개의 선택지 중 하나를 예측하는 이진 분류 문제를 수행하는 모델입니다. 이진 분류 문제의 경우, 출력층에 로지스틱 회귀를 사용해야 하므로 활성화 함수로는 시그모이드 함수를 사용하고, 손실 함수로 크로스 엔트로피 함수를 사용합니다. 하이퍼파라미터인 배치 크기는 64이며, 10 에포크를 수행합니다.

EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)는 검증 데이터 손실(val_loss)이 증가하면, 과적합 징후므로 검증 데이터 손실이 4회 증가하면 정해진 에포크가 도달하지 못하였더라도 학습을 조기 종료(Early Stopping)한다는 의미입니다. ModelCheckpoint를 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델을 저장합니다. validation_split=0.2을 사용하여 훈련 데이터의 20%를 검증 데이터로 분리해서 사용하고, 검증 데이터를 통해서 훈련이 적절히 되고 있는지 확인합니다. 검증 데이터는 기계가 훈련 데이터에 과적합되고 있지는 않은지 확인하기 위한 용도로 사용됩니다.

In [ ]:
z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z = Dropout(dropout_ratio[1])(z)
z = Dense(hidden_units, activation="relu")(z)
model_output = Dense(1, activation="sigmoid")(z)

model = Model(model_input, model_output)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('CNN_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2, verbose=2, callbacks=[es, mc])

학습 후 저장한 모델을 로드하여 테스트 데이터에 대해서 평가합니다.

In [ ]:
loaded_model = load_model('CNN_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

참고자료:

http://www.joshuakim.io/understanding-how-convolutional-neural-network-cnn-perform-text-classification-with-word-embeddings/

https://www.datasciencecentral.com/profiles/blogs/sentence-classification-using-cnns

https://towardsdatascience.com/character-level-cnn-with-keras-50391c3adf33

